# Sentinel-3 OLCI RGB plotter

    Version: 2.0
    Date:    17/09/2019
    Author:  Ben Loveday (Plymouth Marine Laboratory ) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

**What is this notebook for?**

This notebook shows you how to download an OLCI EFR (full-resolution level-1) scene using the harmonised data access API and plot it to the screen. It will walk you through some options for how handle the data, how to convert the radiometry channels to RGB, and how to re-project and plot. It also provides some tricks and tips for plotting imagery of this kind - such as how to make the image more visually appealing. Although it is developed specifically for OLCI, you could use the basis of this script for any RGB channel data (e.g. Sentinel-2 MSI, Sentinel-3 SLSTR, Sentinel-1)

**What specific tools does this notebook use?**

Beyond standard tools, the notebook imports some functions for managing the harmonised data access api (harmonised_data_access_api_tools.py) and some functions for helping us to plot data (image_tools.py).

***

Lets begin....

Python is divided into a series of modules that each contain a series of methods for specific tasks. The box below imports all of the moduls we need to complete our plotting task

In [ ]:
%matplotlib inline

# standard tools
import os
import sys
import json
import xarray as xr
import numpy as np
from zipfile import ZipFile
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from skimage import exposure
from IPython.core.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# specific tools (which can be found here ../Hub_tools/)
sys.path.append(os.path.dirname(os.getcwd()) + '/Hub_Tools/')
import harmonised_data_access_api_tools as hapi
import image_tools as img

WEkEO provides access to a huge number of datasets through its 'harmonised-data-access' API. This allows us to query the full data catalogue and download data quickly and directly onto our Jupyter Hub. You can search for what data is available here: https://www.wekeo.eu/dataset-navigator/start.

In order to use the HDA-API we need to provide some authentication credentials, which comes in the form of an api_key. You can get your key from here; https://www.wekeo.eu/api-keys. If you click on the 'show hidden keys' button at the bottom of the page it will reveal a number of keys. The one you need is in the top grey box, and is on the following line:

-H "Authorization: Basic "**YOUR API KEY**"

Replace "YOUR API KEY" below with what you copy from "**YOUR API KEY**" (N.B. you need to keep the quotation marks.)

We will also define a few other parameters including where to download the data to, and if we want the HDA-API functions to be verbose. **Lastly, we will tell the notebook where to find the query we will use to find the data.** These 'JSON' queries are what we use to ask WEkEO for data. They have a very specific form, but allow us quite fine grained control over what data to get. You can find the example one that we will use here: **JSON_templates/RGB/EO_EUM_DAT_SENTINEL-3_OL_1_EFR___.json**

In [ ]:
# your api key:
api_key = "cmJ1UGJQVzZnT09HU2RUWDJhTGFkOGY4RjhnYTpGRmFCTTNoSXluVk1NdEk4b2dPc2ZjMHFOdlVh"
# where the data should be downloaded to:
download_dir_path = "/home/jovyan/work/products"
# where we can find our data query form:
JSON_query_dir = os.path.join(os.getcwd(),'JSON_templates','RGB')
# HDA-API loud and noisy?
verbose = False

Now that is done, we are going to make some choices about what we should do to our image once we have our data. There are a few operations that we are able to test with respect to images.

    1. truncating the image channel extents
        e.g. should we cut off the brightest and darkest pixels?
        
    2. normalising the channels as a group, or individually (which I call unhitched)
        e.g. how do we map our red/green/intensity to values between 0 and 1
        
    3. histogramming the image channels to reduce the dynamic range
        e.g. do we want to 'squash' the dynamic range of our plot to pull our certain features

    4. Do we want to change the "contrast" and "brightness" (or at least rough proxies for these),  
       across our image?

You might want to test a number of options on your own specific image. This can be time consuming on large data. To make this a bit easier, there are also options for subsetting your image and re-sampling your image at coarses resolution. 

In [ ]:
# image reduction settings: resample the image every grid_factor points
reduce_image = False
grid_factor = 5

# subset image: cut a relevant section out of an image. subset_extents [lon1,lon2,lat1,lat2] describes the section.
subset_image = True
subset_extents = [17.0, 22.0, 57.0, 60.0]

# image truncation settings
truncate_image = True
min_percentile = 5
max_percentile = 95

# image normalisation settings
unhitch = True
channel_contrast = [1.0, 1.0, 1.0] # r,g,b
channel_brightness = 1.0

# image histogram settings
histogram_image = True
histogram_channels = 512

# image plotting settings: e.g. fontsize (fsz)
fsz = 20

Now we have set how we want the script to run, we are ready to get some data. We start this process by telling the script what kind of data we want. In this case, this is OLCI L1 data, which has the following designation on WEkEO: **EO:EUM:DAT:SENTINEL-3:OL_1_EFR___**.

In [ ]:
# OLCI FULL RESOLUTION L1 FILE
dataset_id = "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___"

Here, we use this dataset_id to find the correct, locally stored JSON query file which describes the data we want. The query file is called: **JSON_templates/RGB/EO_EUM_DAT_SENTINEL-3_OL_1_EFR___.json**

You can edit this query if you want to get different data, but be aware of asking for too much data - you could be here a while. The box below gets the correct query file.

In [ ]:
# find query file
JSON_query_file = os.path.join(JSON_query_dir,dataset_id.replace(':','_')+".json")
if not os.path.exists(JSON_query_file):
    print('Query file ' + JSON_query_file + ' does not exist')
else:
    print('Found JSON query file for '+dataset_id)

Now we have a query, we need to launch it to WEkEO to get our data. The box below takes care of this through the following steps:
    1. initialise our HDA-API
    2. get an access token for our data
    3. accepts the WEkEO terms and conditions
    4. loads our JSON query into memory
    5. launches our search
    6. waits for our search results
    7. gets our result list
    8. gets our download links
    9. downloads our data

This is quite a complex process, so much of the functionality has been buried 'behind the scenes'. If you want more information, you can check out the **harmonised_data_access_api_tools.py** python script, or the **How_To_Guide-Harmonized_Data_Access-v0.1.3.ipynb** notebook in the samples directory. The code below will report some information as it runs. At the end, it should tell you that one product has been downloaded.

In [ ]:
HAPI_dict = hapi.init(dataset_id, api_key, download_dir_path, verbose=verbose)
HAPI_dict = hapi.get_access_token(HAPI_dict)
HAPI_dict = hapi.accept_TandC(HAPI_dict)

# load the query
with open(JSON_query_file, 'r') as f:
    query = json.load(f)

# launch job
HAPI_dict = hapi.launch_query(HAPI_dict, query)

# wait for jobs to complete
hapi.check_job_status(HAPI_dict)

# check results
HAPI_dict = hapi.get_results_list(HAPI_dict)
HAPI_dict = hapi.get_download_links(HAPI_dict)

# download data
HAPI_dict = hapi.download_data(HAPI_dict, skip_existing=True)

Sentinel data is usually distributed as a zip file, which contains the SAFE format data within. To use this, we must unzip the file. The bow below handles this.

In [ ]:
# unzip file
for filename in HAPI_dict['filenames']:
    if os.path.splitext(filename)[-1] == '.zip':
        print('Unzipping file')
        with ZipFile(filename, 'r') as zipObj:
            # Extract all the contents of zip file in current directory
            zipObj.extractall(os.path.dirname(filename))

Now we have a local data file we can start to read it in. We begin by reading in the spatial grid variables (e.g. latitude and longitude). 

*(N.B. For OLCI, latititude and longitude are stored in a different file to the radiometry. You can find more information on the format of OLCI data by clicking on the **Sentinel-3 Marine User Handbook** via the following link: https://www.eumetsat.int/website/home/Satellites/CurrentSatellites/Sentinel3/OceanColourServices/index.html)*

In [ ]:
unzipped_file = HAPI_dict['filenames'][0].replace('.zip','.SEN3')
ds1 = xr.open_dataset(os.path.join(unzipped_file, 'geo_coordinates.nc'))
raster_lat = ds1.latitude.data
raster_lon = ds1.longitude.data
ds1.close()

Now we read in the radiance values. To match OLCI's radiometry to what our eye sees, we need to map the radiance to channels to a red, green and blue profile that approximates what our eyes see. We do this using a mapping called 'Tristimulus' (https://www.britannica.com/science/tristimulus-system). OLCI has 21 radiance channels, but we only need the first 11 here, so lets get those...

In [ ]:
num_channels = 11

if 'EFR' in unzipped_file:
    radiometry_type = 'Oa%s_radiance'
else:
    radiometry_type = 'Oa%s_reflectance'

for rad_channel_number in range(1, num_channels+1):
    rad_channel = radiometry_type % (str(rad_channel_number).zfill(2))
    rad_file = os.path.join(unzipped_file, rad_channel + '.nc') 
    rad_fid = xr.open_dataset(rad_file)
    exec("Ch%s = rad_fid.%s.data" % (str(rad_channel_number).zfill(2),rad_channel))
    rad_fid.close()

Now we use the Tristimulus coefficients for OLCI to map the radiances to red, green and blue channels.

In [ ]:
red = np.log10(1.0 + 0.01 * Ch01 + 0.09 * Ch02 + 0.35 * Ch03 + 0.04 * Ch04 + 0.01 * Ch05 + 0.59 * Ch06 + 0.85 * Ch07 + 0.12 * Ch08 + 0.07 * Ch09 + 0.04 * Ch10)
green = np.log10(1.0 + 0.26 * Ch03 + 0.21 * Ch04 + 0.50 * Ch05 + Ch06 + 0.38 * Ch07 + 0.04 * Ch08 + 0.03 * Ch09 + 0.02 * Ch10)
blue = np.log10(1.0 + 0.07 * Ch01 + 0.28 * Ch02 + 1.77 * Ch03 + 0.47 * Ch04 + 0.16 * Ch05)

Now we have our RGB channels, we can manipulate them for the sake of plotting. The boxes below will run **ONLY** if you set the required tag to **True** above. Subset and reduce have been described above. Truncate_image will, by default find the pixels that are darker/lighter than 5%/95% of the image and set them to the 5%/95% value. This stops very bright/dark pixels from dominating our colour range.

In [ ]:
if subset_image:
    i1, i2, j1, j2 = img.subset_image(raster_lat, raster_lon, subset_extents)
    raster_lat = raster_lat[i1:i2,j1:j2]
    raster_lon = raster_lon[i1:i2,j1:j2]
    red = red[i1:i2,j1:j2]
    green = green[i1:i2,j1:j2]
    blue = blue[i1:i2,j1:j2]

In [ ]:
if reduce_image:
    raster_lat = img.reduce_image(raster_lat, grid_factor=grid_factor)
    raster_lon = img.reduce_image(raster_lon, grid_factor=grid_factor)
    red = img.reduce_image(red, grid_factor=grid_factor)
    green = img.reduce_image(green, grid_factor=grid_factor)
    blue = img.reduce_image(blue, grid_factor=grid_factor)

In [ ]:
if truncate_image:
    red = img.truncate_image(red)
    green = img.truncate_image(green)
    blue = img.truncate_image(blue)

Before we go any further, we are going to "stack" our RGB channels into a single image array

In [ ]:
height = np.shape(red)[0]
width = np.shape(red)[1]
image_array = np.zeros((height, width, 3), dtype=np.float32)

image_array[..., 0] = red
image_array[..., 1] = green
image_array[..., 2] = blue

Now we normalise the image. We have to do this so ensure that we can map the luminosity values for each channel to values between 0 and 1 so python can map the numbers to a colour. However, we can do this either by separating the channels (unhitch) or by considering all channels together. By unhitching, we can underplay the dominance of the blue channel in L1 products. *N.B. we are really starting to drift away from 'true colour' now.* The box below will normalise our image array.

In [ ]:
image_array = img.norm_image(image_array, contrast=channel_contrast, unhitch=unhitch)

Now we can apply a histogram to the image, which may improve our image a little more. *N.B. Be aware that older version of skimage may return errors at this point; you may need to upgrade.*

In [ ]:
if histogram_image:
    image_array = exposure.equalize_adapthist(image_array, nbins=histogram_channels)

Now we are going to map our image to a colour array which we will use to plot our scene.

In [ ]:
mesh_rgb = image_array[:, :-1, :]
colorTuple = mesh_rgb.reshape((mesh_rgb.shape[0] * mesh_rgb.shape[1]), 3)
colorTuple = np.insert(colorTuple, 3, 1.0, axis=1)

If we wanted to just plot an image, without any georeferencing or mapping, we can do this from here using plt.imshow(). But our goal is to add mapping etc., so instead we are going to use plt.pcolormesh(), which we can geolocate on a pixel-by-pixel basis. 

However, If we try to plot using the native projection it becomes problematic as our pixels are not regularly shaped. This can result in white line artefacts in our image. To avoid this problem, we reproject the data to a more regular projection. Here we use the Mercator projection which, even though it is not ideal, is currently the only projection apart form platecaree (lat/lon) that supports gridlines in cartopy (our mapping toolkit).

The box below will take care of all the plotting. There are a great many options to set here, so please have a play and see what you can do!

In [ ]:
# get our land mask from NaturalEarth
land_resolution = '10m'
land_poly = cfeature.NaturalEarthFeature('physical', 'land', land_resolution,
                                    edgecolor='k',
                                    facecolor=cfeature.COLORS['land'])

# intitialise our figure
fig1 = plt.figure(figsize=(20, 20), dpi=300)
plt.rc('font', size=fsz)
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

# make an axis
gs = gridspec.GridSpec(1, 1)
m = plt.subplot(gs[0,0], projection=ccrs.Mercator())

# plot the data
plot1 = m.pcolormesh(raster_lon, raster_lat, \
                     red * np.nan, color=colorTuple ** channel_brightness, \
                     clip_on = True,
                     edgecolors=None, zorder=0, \
                     transform=ccrs.PlateCarree())

# change the plot extent if required
if subset_image:
    m.set_extent(subset_extents, crs=ccrs.PlateCarree())

# embellish with gridlines and ticks
g1 = m.gridlines(draw_labels = True, zorder=20, color='0.5', linestyle='--',linewidth=0.5)
g1.xlocator = mticker.FixedLocator(np.linspace(int(subset_extents[0]),\
                                               int(subset_extents[1]), 5))
g1.ylocator = mticker.FixedLocator(np.linspace(int(subset_extents[2]),\
                                               int(subset_extents[3]), 5))
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': fsz, 'color': 'black'}
g1.ylabel_style = {'size': fsz, 'color': 'black'}

With a bit of luck, you now have a wonderful image of a cyanobacterial bloom in the Baltic Sea! You can find more information on this image here: https://www.eumetsat.int/website/home/Images/ImageLibrary/DAT_4574832.html.

If you like, now try to run this script on any other OLCI L1 products, or adapt it for other products. Good luck!